In [14]:
from langchain_community.document_loaders import Docx2txtLoader #To load word files
from langchain.document_loaders import PyPDFLoader #To load pdf files
from langchain.vectorstores import Chroma #Vectorial DataBase
from langchain.embeddings.openai import OpenAIEmbeddings #Word2Vec Model to tokenizer the text
from langchain.text_splitter import RecursiveCharacterTextSplitter #To splitt the text
from langchain.chat_models import ChatOpenAI #To load LLM from OPENAI
from langchain.chains import RetrievalQA #To build chains to QA tasks
from langchain import PromptTemplate #Class that allow the before to get in in production
from langchain.chains.summarize import load_summarize_chain #To buil chains to summarize tasks
import os
from langchain.vectorstores import Chroma #The vectorial database
from langchain_community.document_loaders import TextLoader

DEFINIMOS LA CARPETA CON LOS ARCHIVOS A CODIFICAR

In [6]:
#CARRY THE TXT FILES CONTENT TO LANGCHAIN FORMAT:
concatenated_files=[]
root_path="FilesData/"
txt_files=list(os.listdir("FilesData"))
for txt in txt_files:
    loader = TextLoader(root_path+txt)
    data=loader.load()
    concatenated_files.extend(data)

In [7]:
concatenated_files ## [elementos con 2 atributos page_content, route_path]

[Document(page_content='Nombrar al mejor jugador de la historia del fÃºtbol es un tema muy debatido y subjetivo, ya que depende de diversos factores como la Ã©poca, el estilo de juego, los logros individuales y colectivos, entre otros. Sin embargo, uno de los nombres que suele destacarse con mayor frecuencia es el de PelÃ©, el legendario futbolista brasileÃ±o.\n\nPelÃ©, cuyo nombre real es Edson Arantes do Nascimento, naciÃ³ el 23 de octubre de 1940 en TrÃªs CoraÃ§Ãµes, Brasil. Es ampliamente considerado como uno de los jugadores mÃ¡s talentosos y exitosos en la historia del fÃºtbol. Su habilidad tÃ©cnica, su visiÃ³n de juego, su capacidad goleadora y su liderazgo en el campo lo convierten en una figura legendaria.\n\nPelÃ© alcanzÃ³ la fama internacional durante la dÃ©cada de 1950 y 1960, cuando liderÃ³ a la selecciÃ³n brasileÃ±a a la conquista de tres Copas del Mundo de la FIFA en 1958, 1962 y 1970. En estas competiciones, PelÃ© demostrÃ³ su genio futbolÃ\xadstico, anotando goles memo

ARMAMOS SET DE DATOS PARA EVITAR SATURAR LA VECTORIZACIÓN

In [8]:
#As we can see, the LLM can procces a limit amount of tokens, so that we have to split the text in fragments of 1500 tokens in this case (because is the maximun amount of tokens that support our model)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, #Fragments of text of 1500 tokens
    chunk_overlap=200, #For evey fragment that take the 200 last tokens of the last fragment
    length_function=len
    )

documents = text_splitter.split_documents(concatenated_files) #List with the metadata and the content splitt by fragments of 1500 tokens

In [9]:
documents

[Document(page_content='Nombrar al mejor jugador de la historia del fÃºtbol es un tema muy debatido y subjetivo, ya que depende de diversos factores como la Ã©poca, el estilo de juego, los logros individuales y colectivos, entre otros. Sin embargo, uno de los nombres que suele destacarse con mayor frecuencia es el de PelÃ©, el legendario futbolista brasileÃ±o.\n\nPelÃ©, cuyo nombre real es Edson Arantes do Nascimento, naciÃ³ el 23 de octubre de 1940 en TrÃªs CoraÃ§Ãµes, Brasil. Es ampliamente considerado como uno de los jugadores mÃ¡s talentosos y exitosos en la historia del fÃºtbol. Su habilidad tÃ©cnica, su visiÃ³n de juego, su capacidad goleadora y su liderazgo en el campo lo convierten en una figura legendaria.\n\nPelÃ© alcanzÃ³ la fama internacional durante la dÃ©cada de 1950 y 1960, cuando liderÃ³ a la selecciÃ³n brasileÃ±a a la conquista de tres Copas del Mundo de la FIFA en 1958, 1962 y 1970. En estas competiciones, PelÃ© demostrÃ³ su genio futbolÃ\xadstico, anotando goles memo

## GUARDAMOS COMO UNA VARIABLE DE ENTORNO LA API KEY

In [10]:
OPENAI_API_KEY = "YOUR_OPENAI_API_KEY"
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## CARGAMOS EL MODELO

In [12]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002") #word2vec model of openAI

In [16]:
NOMBRE_INDICE_CHROMA = "archivos_pruebas" #Name of my vectorial database (Put the name that you want)

#Creating our vectorial database or vector store
vectorstore_chroma = Chroma.from_documents(
    documents=documents, #Create the database with the list of the created documents (Every instance will be the embedding of every document)
    embedding=embeddings, #Word2vec model to create our embeddings, always use the same.
    persist_directory=NOMBRE_INDICE_CHROMA #Load my database in the indicated folder (If I close the section, I will keep storaged my vectorial databas in the folder called "NOMBRE_INDICE_CHROMA" )
)

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.